In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/mac/Downloads/주문내역_51174_2025-06-01~2025-06-13.xlsx')
ka_df = pd.read_excel('/Users/mac/Downloads/20250613141530.xlsx')

In [3]:
fs_df['옵션명'].unique()

array(['4박스 (8개월분) (28%)', '6박스(12개월분) (35%)', '2박스 (4개월분) (20%)'],
      dtype=object)

In [4]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)

In [5]:
fs_df['주문 상태'].unique()

array(['결제완료', '배송중', '취소완료', '배송완료', '구매확정'], dtype=object)

In [6]:
ka_df['박스수량'] = ka_df['옵션'].apply(lambda x: x.split('박스')[0].split(': ')[1])
ka_df['박스수량'] = ka_df['박스수량'].astype(int)

In [7]:
ka_df['박스수량'].unique()

array([2, 6, 4])

In [8]:
fs_df['구분'] = '친한스토어'
ka_df['구분'] = '톡스토어'

In [9]:
fs_df = fs_df[['결제 번호', '주문 상태','결제 완료일', '옵션명', '수량', '박스수량', '정산가', '구분']]

In [10]:
fs_df['옵션명'].unique()

array(['4박스 (8개월분) (28%)', '6박스(12개월분) (35%)', '2박스 (4개월분) (20%)'],
      dtype=object)

In [11]:
ka_df = ka_df[['결제번호', '주문상태', '발송요청일', '옵션', '수량', '박스수량', '정산기준금액', '구분']]

In [12]:
con1 = ka_df['옵션'] == '선택: 2박스 (4개월분) (20%)'
con2 = ka_df['옵션'] == '선택: 4박스 (8개월분) (28%)'
con3 = ka_df['옵션'] == '선택: 6박스(12개월분) (35%)'

ka_df.loc[con1, '옵션'] = '2박스 (4개월분) (20%)'
ka_df.loc[con2, '옵션'] = '4박스 (8개월분) (28%)'
ka_df.loc[con3, '옵션'] = '6박스(12개월분) (35%)'

In [13]:
ka_df.columns = fs_df.columns

In [14]:
df = pd.concat([fs_df, ka_df])

In [15]:
df.columns = ['결제 번호', '주문 상태', '결제 완료일', '옵션명', '수량', '박스수량', '매출', '구분']

In [16]:
df['주문 상태'].unique()

array(['결제완료', '배송중', '취소완료', '배송완료', '구매확정', '304 배송 중', '305 배송 완료',
       '601 구매 결정', '303 결제 취소 완료'], dtype=object)

In [17]:
status = ['결제완료', '배송준비', '배송완료', '배송중', '구매확정', '202 배송 요청', '304 배송 중', '305 배송 완료', '601 구매 결정']

In [21]:
cancel_df = df[~df['주문 상태'].isin(status)]

,결제 번호,주문 상태,결제 완료일,옵션명,수량,박스수량,매출,구분


In [18]:
df = df[df['주문 상태'].isin(status)]

In [19]:
df['주문 상태'].unique()

array(['결제완료', '배송중', '배송완료', '구매확정', '304 배송 중', '305 배송 완료',
       '601 구매 결정'], dtype=object)

In [20]:
df['결제 완료일'] = pd.to_datetime(df['결제 완료일'])
df['결제 완료일'] = df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pivot_df = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '박스수량'], aggfunc=['sum'], fill_value=0, margins=True, margins_name='총합')
pivot_df.columns = pivot_df.columns.droplevel(0)
temp_df = pd.DataFrame(pivot_df)

pivot = pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')
temp_df2 = pd.DataFrame(pivot)

result_df = pd.merge(temp_df, temp_df2, left_index=True, right_index=True, how='inner')
result_df.rename(columns={'결제 번호': '주문수량'}, inplace=True)
result_df = result_df.applymap(lambda x: f'{x:,}')
final_df = pd.DataFrame(result_df)
#final_df.to_excel(f'/Users/mac/Downloads/{datetime.today()}약사비책님.xlsx')


In [22]:
final_df

매출 박스수량 주문수량
결제 완료일     옵션명                                   
2025-06-09 2박스 (4개월분) (20%)   3,784,000   86   43
           4박스 (8개월분) (28%)   3,484,800   88   22
           6박스(12개월분) (35%)   1,501,500   42    7
2025-06-10 2박스 (4개월분) (20%)     616,000   14    7
           4박스 (8개월분) (28%)   1,108,800   28    7
           6박스(12개월분) (35%)     858,000   24    4
2025-06-11 2박스 (4개월분) (20%)   1,056,000   24   12
           4박스 (8개월분) (28%)     950,400   24    6
           6박스(12개월분) (35%)     429,000   12    2
2025-06-12 2박스 (4개월분) (20%)     528,000   12    6
           4박스 (8개월분) (28%)     316,800    8    2
           6박스(12개월분) (35%)     214,500    6    1
2025-06-13 2박스 (4개월분) (20%)      88,000    2    1
           4박스 (8개월분) (28%)     475,200   12    3
           6박스(12개월분) (35%)     214,500    6    1
총합                           15,625,500  388  124

In [23]:
temp_df3 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values=['매출', '박스수량'], aggfunc='sum'))
temp = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '구분'], values='결제 번호', aggfunc='count'))
temp.rename(columns={'결제 번호': '주문수량'}, inplace=True)
col = temp['주문수량']
temp_df3['주문수량'] = col
temp_df3.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
결제 완료일     구분                        
2025-06-09 친한스토어  5,894,900  146   47
           톡스토어   2,875,400   70   25
2025-06-10 친한스토어  1,502,600   38   11
           톡스토어   1,080,200   28    7
2025-06-11 친한스토어  1,696,200   42   14
           톡스토어     739,200   18    6
2025-06-12 친한스토어    580,800   14    5
           톡스토어     478,500   12    4
2025-06-13 친한스토어    777,700   20    5

In [24]:
#df['제형'] = df['옵션명'].apply(lambda x: x.split('_')[0].split('(')[1])
#df['제형'] = df['제형'].apply(lambda x: x[:2])
#temp_df1 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['매출', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합'))
#temp_df2 = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '제형'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합'))
#temp_df2.columns = ['주문수량']

#result = pd.merge(temp_df1, temp_df2, left_index=True, right_index=True, how='inner')
#result.applymap(lambda x: f'{x:,}')